## 台灣證劵交易所（個股日成交公開資訊）

* **[Plotly 下載官網](https://pypi.org/project/plotly/)**
* **Plotly 教學 [[1]](https://plotly.com/python/) [[2]](https://blog.csdn.net/u012897374/article/details/77857980) [[3]](https://ironpdf.com/zh-hant/python/blog/python-help/plotly-python/)**
* **Plotly Express 介紹 [[1]](https://ithelp.ithome.com.tw/articles/10277258) [[2]](https://plotly.com/python/plotly-express/)**
* **[在 Anaconda 中裝 Plotly](https://plotly.com/python/getting-started/)**
* **[Python Plotly 資料視覺化 & 圖表互動的必備利器，新手必學懶人包](https://igoamazing.com/python-plotly/)**

In [ ]:
!pip install plotly==5.10.0 --user

In [ ]:
!pip install --upgrade plotly --user    # 裝最新版，與 5.10.0 版二選一

***

In [ ]:
!pip install jupyter anywidget --user    # 選項（可不裝）

In [ ]:
!pip install -U kaleido --user    # 選項（可不裝）

***

In [ ]:
# %load stockyear_plotly.py

#
# 可以去掉 Python 輸出時，因為軟體版本所引起的警告的警告。
#
import warnings
warnings.filterwarnings('ignore')

#
# 匯入本範例必要的套件
#
import requests
import json, csv
import pandas as pd
import os
import time
from random import randint
import plotly
from plotly.graph_objs import Scatter, Layout


#
# 偽裝成瀏覽器，因此利用使用者自訂 http browser request header 中的 user-agent 內容的方式來實現
#
headers = { 
            'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64)\
             AppleWebKit/537.36 (KHTML, like Gecko)\
             Chrome/83.0.4103.61 Safari/537.36', 
          }       # '\'為續行字符，代表與下一行為同一行的敘述


def twodigit(n):  #將數值轉為二位數字串
    
    if(n < 10):
        
        retstr = '0' + str(n)
        
    else:
        
        retstr = str(n)
        
    return retstr


def convertDate(date):  # 轉換民國日期為西元，例如： 108/03/02->20190302

    str1 = str(date)
    
    yearstr = str1[:3]  # 取出民國年
    
    realyear = str(int(yearstr) + 1911)  # 轉為西元年
    
    realdate = realyear + str1[4:6] + str1[7:9]  # 組合日期
    
    return realdate


#
# 當使用 plotly.offline.iplot 在 Jupyter筆記本中離線繪圖時，
# 所需處理一個額外的初始化步驟。請參考 https://plot.ly/python/offline/
#
plotly.offline.init_notebook_mode(connected = True) 
    
#
# 取消顯示 Pandas 資料重設警告，這個設置會關閉掉 copywarning，
# 也有人提到關閉這個 warning後，速度更快，有待驗證。
#
pd.options.mode.chained_assignment = None  

### 本範例欲爬的目標網站：[台灣證劵交易所>>首頁>>交易資訊>>盤後資訊>>個股日成交資訊](https://www.twse.com.tw/zh/page/trading/exchange/STOCK_DAY.html)

In [ ]:
#
# 以下區域變數內容可以自行設定
#

year = 2024      # 欲擷取資料的年份

stockNo = 2317   # 鴻海精密

startMonth = 1   # 欲擷取資料的起始月份

endMonth = 12    # 欲擷取資料的結束月份


#=====================================

#
# 因為證交所的資料是用 json 格式來傳遞
# 所以這一部分是鎖定用 json 的處理方式
# 來完成
#

# 目標網址前半
urlbase = 'https://www.twse.com.tw/rwd/zh/afterTrading/STOCK_DAY?date=' + str(year)  

# 目標網址後半
urltail = '01&stockNo=' + str(stockNo) + '&response=json'  

# 擷取資料後的存檔檔案名稱 (預設在與本程式檔案相同的路徑之下)
filename = str(stockNo) + '_stockyear' + str(year) + '.csv'   


#=====================================

#
# 如果之前的資料檔案存在就自動刪除，以方便建立檔案
#
try:
    
    os.remove(filename)
    
except OSError:
    
    pass


#
# 底下這一行的寫法亦可
#
#os.remove(filename) if os.path.exists(filename) else None  
 
#=====================================

for i in range(startMonth, endMonth + 1):  # 取 1 到 12 數字
            
    url_twse = urlbase + twodigit(i) + urltail  # 組合網址
        
    res = requests.get(url_twse, headers = headers)    # 回傳為 json 資料
        
    jdata = json.loads(res.text)    # json 解析
                       
    outputfile = open(filename, 'a', newline = '', encoding = 'utf-8')  # 開啟資料儲存檔案
        
    outputwriter = csv.writer(outputfile)  # 以 csv 格式寫入檔案
        
    if i == 1:  # 若是 1 月 就寫入欄位名稱
        
        outputwriter.writerow(jdata['fields'])
        
    for dataline in (jdata['data']):  # 逐月寫入資料
            
        outputwriter.writerow(dataline)
        
        
    time.sleep(randint(3, 6))  # 延遲 3 秒以上，否則有時會有錯誤
    
outputfile.close()  # 關閉檔案

#### Plotly - Online and Offline Plotting [[1]](https://www.tutorialspoint.com/plotly/plotly_online_and_offline_plotting.htm) [[2]](https://plotly.com/python/) [[3]](https://medium.com/ichitsai/vis-plot-ly-offline-python-%E8%B3%87%E6%96%99%E8%A6%96%E8%A6%BA%E5%8C%96-f4b540c130f8) [[4]](https://blog.csdn.net/u012897374/article/details/77857980)

In [ ]:
pdstock = pd.read_csv(filename, encoding = 'utf-8')  # 以 Pandas 讀取檔案

#
# # 轉換日期表示式為西元年格式
#
for i in range(len(pdstock['日期'])):

    pdstock['日期'][i] = convertDate(pdstock['日期'][i])


#
# 轉換日期欄位為日期格式
#
pdstock['日期'] = pd.to_datetime(pdstock['日期'])


data = [
    Scatter(x = pdstock['日期'], y = pdstock['收盤價'], name = '收盤價'),
    Scatter(x = pdstock['日期'], y = pdstock['最低價'], name = '最低價'),
    Scatter(x = pdstock['日期'], y = pdstock['最高價'], name = '最高價')
]


plotly.offline.iplot({  # 以 plotly 繪圖
    "data": data,
    "layout": Layout(title = str(year) + '年個股統計圖')
}) 